## Advanced Lane Finding Project

The goals / steps of this project are the following:

* DONE Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* DONE Apply a distortion correction to raw images.
* DONE Use color transforms, gradients, etc., to create a thresholded binary image.
* DONE Apply a perspective transform to rectify binary image ("birds-eye view").
* DONE Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* DONE Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

# Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib
matplotlib.use('Qt4Agg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#%matplotlib qt

In [ ]:
def Calibrate():
    nx = 9
    ny = 6

    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((ny*nx,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
#            cv2.imshow('img',img)
#            cv2.imwrite(fname+"_corners.png", img);
#            cv2.waitKey(500)

#    cv2.destroyAllWindows()

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
    return ret, mtx, dist, rvecs, tvecs

# Apply a distortion correction to raw images.

In [ ]:
def cal_undistort(img):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

# Use color transforms, gradients, etc., to create a thresholded binary image.

In [ ]:
def abs_sobel_thresh(img, orient='x', sobel_kernel=9, thresh = (0,255), channel = "grey"):
    if channel == 'grey':
        # Convert to grayscale
        channel = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if channel == 's':
        hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
        channel = hls[:,:,2]
    if channel == 'v':
        luv = cv2.cvtColor(img, cv2.COLOR_BGR2Luv)
        channel = luv[:,:,2]
    if channel == 'r':
        channel = img[:,:,2]
        
    # Take the Sobel x or y gradient
    sobel = cv2.Sobel(channel, cv2.CV_64F, (orient == 'x'), (orient == 'y'), ksize=sobel_kernel)
    # Take the absolute value of the gradient
    abs_sobel = np.absolute(sobel)
    # Rescale to 8 bit
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    # Return the binary image
    return binary_output

def mag_thresh(img, sobel_kernel=9, thresh=(30, 100), channel = "grey"):
    if channel == 'grey':
        # Convert to grayscale
        channel = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if channel == 's':
        hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
        channel = hls[:,:,2]
    if channel == 'v':
        luv = cv2.cvtColor(img, cv2.COLOR_BGR2Luv)
        channel = luv[:,:,2]
    if channel == 'r':
        channel = img[:,:,2]
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(channel, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(channel, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    gradmag = (255*gradmag/np.max(gradmag)).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= thresh[0]) & (gradmag <= thresh[1])] = 1

    # Return the binary image
    return binary_output

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

def color_threshold(img, thresh = ([170,170,170], [255,255,255]), color_conv = 0, Show = False):

    conv_img = np.copy(img)

    # Convert to desired color space, or leave as BGR
    if color_conv != 0:
        conv_img = cv2.cvtColor(img, color_conv)
        
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_thresh = np.zeros_like(conv_img)
    binary_thresh[(conv_img > thresh[0]) & (conv_img <= thresh[1])] = 1

    # Stack the threshold channels into a color image
    binary_output = (binary_thresh[:,:,0] | binary_thresh[:,:,1] | binary_thresh[:,:,2])
    
    return binary_output


# Edit this function to create your own pipeline.
def thresh_pipeline(img, Show = False):    
    # Create a copy of the image
    img = np.copy(img)
    # Choose a Sobel kernel size
    ksize = 3
    # Apply each of the thresholding functions
    Sat_bin = color_threshold(img, thresh = ([0,0,100],[0,0,200]), color_conv = cv2.COLOR_BGR2HLS)
    HSV_bin = color_threshold(img, thresh = ([0,150,200],[0,255,255]), color_conv = cv2.COLOR_BGR2HSV)
    Val_bin = color_threshold(img, thresh = ([0,0,200],[0,0,255]), color_conv = cv2.COLOR_BGR2HSV)
    Red_bin = color_threshold(img, thresh = ([0,0,175],[0,0,255]))
    Luv_mag_bin = mag_thresh(img, thresh=(60,255), channel = 'v')
    Grey_x_bin = abs_sobel_thresh(img, orient='x', sobel_kernel=11, thresh=(20, 100), channel = 'grey')
    Dir_bin = dir_threshold(img, sobel_kernel=5, thresh=(0.4, 1.4))
    
    color_bin = np.zeros_like(Dir_bin)
    color_bin[((Sat_bin == 1) | (HSV_bin == 1) | (Val_bin == 1) | (Red_bin == 1)) & (Dir_bin == 1)] = 1
    
    grad_bin = np.zeros_like(Dir_bin)
    grad_bin[(Grey_x_bin == 1) & (Dir_bin == 1)] = 1

    yellow_bin = np.zeros_like(Dir_bin)
    yellow_bin[((Luv_mag_bin == 1) & (Dir_bin == 1))] = 1

    # Combine the thresholds
    combined_binary = np.zeros_like(Dir_bin)
    combined_binary[((color_bin == 1) & (grad_bin == 1)) | (yellow_bin == 1)] = 1

    if Show:
        plt.figure()
        plt.subplot(221)
        plt.title('color_bin')
        plt.imshow(color_bin*255, cmap = 'gray')
        plt.subplot(222)
        plt.title('yellow_bin')
        plt.imshow(yellow_bin*255, cmap = 'gray')
        plt.subplot(223)
        plt.title('grad_bin')
        plt.imshow(grad_bin*255, cmap = 'gray')
        plt.subplot(224)
        plt.title('combined_binary')
        plt.imshow(combined_binary*255, cmap = 'gray')
        plt.show()
    
    # Return the combined threshold
    return combined_binary.astype(np.uint8)

In [ ]:
images = glob.glob('test_video_frames/tough_frames/*.jpg')
#images = [images[5]]
for fname in images:
    image = cv2.imread(fname, cv2.IMREAD_COLOR)
    undist = cal_undistort(image)
    thresh_pipeline(undist, Show = True)

# Apply a perspective transform to rectify binary image ("birds-eye view").

In [ ]:
def get_warp_matrix(img):
    
    # Compute and apply perpective transform
    dx, dy = img.shape[1], img.shape[0]
    src = np.float32([[714, 466],[1107,dy],[219,dy],[573, 466]])
    dst = np.float32([[dx*3/4,0],[dx*3/4,dy],[dx/4,dy],[dx/4,0]])
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    return M, Minv

def warp(img, M):
    # Compute and apply perpective transform
    warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))  # keep same size as input image    
    return warped

def unwarp(undist, MinV, color_warp):
    # Warp the lane lines back onto the original image
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (color_warp.shape[1], color_warp.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    return result

# Detect lane pixels and fit to find the lane boundary.

In [ ]:
def find_lane_lines(binary_warped, Show = False):
    # This function finds lane lines from scratch
    # If Show is True then it also plots output visualizations
    # This is disabled by default to speed up the algorithm
    
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)
    
    if Show:
        plt.figure()
        plt.plot(histogram)
        plt.show()
        # Create an output image to draw on and  visualize the result
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # Set the width of the windows +/- margin
    margin = 150
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    first_frame = True
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        if first_frame:
            margin = 200
            first_frame = False
        else:
            margin = 150
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        if Show:
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    try:
        left_fit = np.polyfit(lefty, leftx, 2)
    except TypeError:
        left_fit = np.zeros((1,3))
    try:
        right_fit = np.polyfit(righty, rightx, 2)
    except TypeError:
        right_fit = np.zeros((1,3))
        
    if Show:
        plt.figure()
        # Generate x and y values for plotting
        ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
        plt.imshow(out_img)
        plt.show()
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.xlim(0, 1280)
        plt.ylim(720, 0)
    
    return left_fit, right_fit, left_lane_inds, right_lane_inds

def find_next_lane_lines(binary_warped, left_fit, right_fit):
    # takes the binary output from the thresholded image and the previous lane lines
    # returns the new lane predictions

    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
    left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
    right_fit[1]*nonzeroy + right_fit[2] + margin)))      
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit a second order polynomial to each
    try:
        left_fit = np.polyfit(lefty, leftx, 2)
    except TypeError:
        left_fit = np.zeros((1,3))
    try:
        right_fit = np.polyfit(righty, rightx, 2)
    except TypeError:
        right_fit = np.zeros((1,3))

    return left_fit, right_fit, left_lane_inds, right_lane_inds

# Determine the curvature of the lane and vehicle position with respect to center.

In [ ]:
def lane_curvature(binary_warped, fit):
    #Calculate the radius of curvature for the lane
    # fills the road bounded by the two lanes with an overlay
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    fitx = fit[0]*ploty**2 + fit[1]*ploty + fit[2]

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Define y-value where we want radius of curvature
    # I'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)

    # Convert pixels to meters and fit a second order polynomial to each
    fit_cr = np.polyfit(ploty*ym_per_pix, fitx*xm_per_pix, 2)

    # Fit new polynomials to x,y in world space
    # Calculate the new radii of curvature
    curverad = ((1 + (2*fit_cr[0]*y_eval*ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])
    
    # If a radius over 500km is returned, 
    max_curverad = 500000
    curverad = min(curverad,max_curverad)
    return curverad

# Some of the following filtering techniqes were inspired by Paul Heraty
# https://medium.com/@heratypaul/udacity-sdcnd-advanced-lane-finding-45012da5ca7d

def new_lane_detected(binary_warped, ploty, fit):
    max_x_pos = binary_warped.shape[1] # the lane must have an endpoint within the image
    # Assume the lines are not detected until confirmed otherwise
    detected = False

    # Check if the polyfit returned a nonzero value
    if np.amax(fit):
        # Get base position and slope for each line
        x_pos = fit[0]*ploty[-1]**2 + fit[1]*ploty[-1] + fit[2]
        if 0 < x_pos < max_x_pos:
            detected = True
    return detected
    
def next_lane_detected(binary_warped, ploty, left_fit, right_fit):
    # Set max thresholds
    max_pos_delta = 0.1 # Percent delta from last good x position of lane, relative to center of vehicle
    max_curve_delta = 10 # Order of magnitude delta between curvatures
        

    # Assume the lines are not detected until confirmed otherwise
    left_detected, right_detected = False, False

    # Check if the polyfit returned a nonzero value
    if np.amax(left_fit):
        # Get base position and slope for each line
        left_base_pos = left_fit[0]*ploty[-1]**2 + left_fit[1]*ploty[-1] + left_fit[2]
        left_curvature = lane_curvature(binary_warped, left_fit)
        # Determine delta
        left_pos_delta = abs((Left_Line.line_base_pos - left_base_pos)/Left_Line.line_base_pos)
        left_curve_delta = left_curvature/Left_Line.radius_of_curvature
        # Check if thresholds are within bounds
        if (left_pos_delta < max_pos_delta) & ((1/max_curve_delta) < left_curve_delta < max_curve_delta):
            left_detected = True
    
    # Check if the polyfit returned a nonzero value
    if np.amax(right_fit):
        right_base_pos = right_fit[0]*ploty[-1]**2 + right_fit[1]*ploty[-1] + right_fit[2]
        right_curvature = lane_curvature(binary_warped, right_fit)
        # Determine delta
        right_pos_delta = abs((Right_Line.line_base_pos - right_base_pos)/Right_Line.line_base_pos)
        right_curve_delta = right_curvature/Right_Line.radius_of_curvature
        # Check if thresholds are within bounds
        if (right_pos_delta < max_pos_delta) & ((1/max_curve_delta) < right_curve_delta < max_curve_delta):
            right_detected = True

    return left_detected, right_detected

def lanes_concur(left_fit, right_fit, left_curvature, right_curvature, ploty):
#    max_angle_delta = 1 # Slope delta between left and right lane slopes; 45 deg
    max_curve_delta = 10000 # Order of magnitude delta between curvatures
    
    # In the warped frames, the distance between lane lines are almost always between these values
    min_pos_delta = 500
    max_pos_delta = 800

    left_base_pos = left_fit[0]*ploty[-1]**2 + left_fit[1]*ploty[-1] + left_fit[2]
    right_base_pos = right_fit[0]*ploty[-1]**2 + right_fit[1]*ploty[-1] + right_fit[2]
    base_delta = right_base_pos - left_base_pos
    
    lane_curve_delta = abs(left_curvature/right_curvature)
    if ((1/max_curve_delta) < lane_curve_delta < max_curve_delta) & (min_pos_delta < base_delta < max_pos_delta):
        result = True
    else:
        result = False
        print(left_base_pos, right_base_pos, lane_curve_delta, base_delta)
    return result

# Warp the detected lane boundaries back onto the original image.

In [ ]:
# Borrowed from https://stackoverflow.com/questions/35355930/matplotlib-figure-to-image-as-a-numpy-array
from matplotlib.backends.backend_agg import FigureCanvasAgg
from matplotlib.figure import Figure

def draw_lane_lines(binary_warped, left_fit, right_fit, left_lane_inds, right_lane_inds):
    # This should be called on a warped image
    # This function draws bars around the lane lines and highlights the pixels for each lane
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    margin = 100
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                                  ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                                  ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    fig = Figure()
    canvas = FigureCanvasAgg(fig)
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    plt.imshow(result)
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    plt.show()
    
    return result

def draw_lane_boundary(binary_warped, left_fit, right_fit):
    # fills the road bounded by the two lanes with an overlay
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]),(100,100,255))

    return color_warp

# Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [ ]:
# Different lane line pipelines for various outputs

# simple lane line processing without using buffers
def process_image(fname, image):
    undist = cal_undistort(image)
    cv2.imwrite(fname+"_undist.png", undist);
    binary_thresh = thresh_pipeline(undist)
    cv2.imwrite(fname+"_binary_thresh.png", binary_thresh*255);
    binary_warped = warp(binary_thresh, M)
    cv2.imwrite(fname+"_binary_warped.png", binary_warped*255);
    left_fit, right_fit, left_lane_inds, right_lane_inds = find_lane_lines(binary_warped, Show = True)
    lanes_warped = draw_lane_boundary(binary_warped, left_fit, right_fit)
    lanes = unwarp(undist, Minv, lanes_warped)
    cv2.imwrite(fname+"_lanes.png", lanes);
    return

# Advanced lane line processing which utilizes a moving average and plasibility checks to smooth and filter the lanes
def adv_proc_img(image, movie = True):

    # moviepy outputs RGB images, OpenCV uses BGR images
    if movie:
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    undist = cal_undistort(image)
    binary_thresh = thresh_pipeline(undist)
    binary_warped = warp(binary_thresh, M)
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    max_lost_frames = 7
    
    # Check if there has been a detection of either lane in the last 7 frames
    if (Right_Line.last_detected < max_lost_frames) | (Left_Line.last_detected < max_lost_frames):
        # Find the next lane lines for both lanes (may only use one lane though)
        left_fit, right_fit, left_lane_inds, right_lane_inds = find_next_lane_lines(binary_warped, Left_Line.best_fit, Right_Line.best_fit)
    
        # Determine if new lane is a valid lane
        Left_Line.detected, Right_Line.detected = next_lane_detected(binary_warped, ploty, left_fit, right_fit)
        # If the right lane is known, add new lane information
        if Left_Line.detected:
            Left_Line.last_detected = 0
            # If the buffer is full, remove the oldest entries
            if len(Left_Line.current_fit) == max_lost_frames:
                Left_Line.current_fit = np.delete(Left_Line.current_fit, 0, axis = 0)
            # Append the newset entries
            Left_Line.current_fit = np.vstack((Left_Line.current_fit,left_fit))
            #Average the entries across the buffer
            Left_Line.best_fit = np.sum(Left_Line.current_fit, axis = 0)/len(Left_Line.current_fit)
            Left_Line.radius_of_curvature = lane_curvature(binary_warped, Left_Line.best_fit)
            Left_Line.line_base_pos = Left_Line.best_fit[0]*ploty[-1]**2 + Left_Line.best_fit[1]*ploty[-1] + Left_Line.best_fit[2]
            Left_Line.all = left_lane_inds
        
        # If the left line is not detected, increment the last detection counter
        else:
            Left_Line.last_detected += 1
            
        # If the right lane is still known, add new lane information
        if Right_Line.detected:
            Right_Line.last_detected = 0
            # If the buffer is full, remove the oldest entries
            if len(Right_Line.current_fit) == max_lost_frames:
                Right_Line.current_fit = np.delete(Right_Line.current_fit, 0, axis = 0)
            # Now append the newset entries
            Right_Line.current_fit = np.vstack((Right_Line.current_fit,right_fit))
            #Average the entries across the buffer
            Right_Line.best_fit = np.sum(Right_Line.current_fit, axis = 0)/len(Right_Line.current_fit)
            Right_Line.radius_of_curvature = lane_curvature(binary_warped, Right_Line.best_fit)
            Right_Line.line_base_pos = Right_Line.best_fit[0]*ploty[-1]**2 + Right_Line.best_fit[1]*ploty[-1] + Right_Line.best_fit[2]
            Right_Line.all = left_lane_inds

        # If the right line is not detected, increment the last detection counter
        else:
            Right_Line.last_detected += 1

    # Check if either line hasn't been detected in at least 7 iterations and start a new line fit
    if (Right_Line.last_detected == max_lost_frames) | (Left_Line.last_detected == max_lost_frames):
        left_fit, right_fit, left_lane_inds, right_lane_inds = find_lane_lines(binary_warped)
                
        # If the left lane hasn't been detected within the max allowed frames and a new lane was found, set values
        if (Left_Line.last_detected == max_lost_frames):
            if new_lane_detected(binary_warped, ploty, left_fit):
                Left_Line.detected = True
                Left_Line.current_fit = left_fit
                Left_Line.best_fit = left_fit
                Left_Line.last_detected = 0
                Left_Line.line_base_pos = Left_Line.best_fit[0]*ploty[-1]**2 + Left_Line.best_fit[1]*ploty[-1] + Left_Line.best_fit[2]
                Left_Line.radius_of_curvature = lane_curvature(binary_warped, Left_Line.best_fit)
                Left_Line.all = left_lane_inds
            # Otherwise ignore this new run
            else:
                Left_Line.detected = False

        # If the left lane hasn't been detected within the max allowed frames and a new lane was found, set values
        if (Right_Line.last_detected == max_lost_frames):
            if new_lane_detected(binary_warped, ploty, right_fit):
                Right_Line.detected = True
                Right_Line.last_detected = 0
                Right_Line.current_fit = right_fit
                Right_Line.best_fit = right_fit
                Right_Line.line_base_pos = Right_Line.best_fit[0]*ploty[-1]**2 + Right_Line.best_fit[1]*ploty[-1] + Right_Line.best_fit[2]
                Right_Line.radius_of_curvature = lane_curvature(binary_warped, Right_Line.best_fit)
                Right_Line.all = right_lane_inds
            # Otherwise ignore this run
            else:
                Right_Line.detected = False

    # If a lane hasn't been seen within the max allowed frames, don't show the projection
    if ((Left_Line.detected == False) & (Left_Line.detected == max_lost_frames)) | ((Right_Line.detected == False) & (Right_Line.last_detected == max_lost_frames)):
        lanes = undist
    # If the lanes agree, show them
    elif lanes_concur(Left_Line.best_fit, Right_Line.best_fit, Left_Line.radius_of_curvature, Right_Line.radius_of_curvature, ploty):

        if movie == False:
            plt.figure()
            plt.imshow(draw_lane_lines(binary_warped, Left_Line.best_fit, Right_Line.best_fit, Left_Line.all, Right_Line.all))
            plt.show()
            
        # Calculate the vehicle offset from center in meters
        xm_per_pix = 3.7/700 # meters per pixel in x dimension
        vehicle_offset = xm_per_pix*(binary_warped.shape[1]/2 - (Left_Line.line_base_pos + (Right_Line.line_base_pos - Left_Line.line_base_pos)/2))
        # Draw the warped lanes
        lanes_warped = draw_lane_boundary(binary_warped, Left_Line.best_fit, Right_Line.best_fit)
        # Unwarp lanes and overlay on frame
        lanes = unwarp(undist, Minv, lanes_warped)
        # Overlay curvature and vehicle position
        lanes = cv2.putText(lanes,'Left radius: {:.0f} m'.format(Left_Line.radius_of_curvature),(100,100), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
        lanes = cv2.putText(lanes,'Right radius: {:.0f} m'.format(Right_Line.radius_of_curvature),(100,130), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
        lanes = cv2.putText(lanes,'Vehicle offset: {:.2f} m'.format(vehicle_offset),(100,160), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)

    # If the lanes don't agree, don't show the projection
    else:
        lanes = undist
        lanes = cv2.putText(lanes,'LOST LANES',(100,115), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)

    # moviepy outputs RGB images, OpenCV uses BGR images
    if movie:
        lanes = cv2.cvtColor(lanes, cv2.COLOR_BGR2RGB)

    return lanes    

        
# check if a line is found that matches the old line
# if found add the polyfit data to the list
# the average the polyfit data across the last 7 inputs
# if not found, check how many frames in a row we havent seen the data
# if it's been over 7 frames, look for a new line

In [ ]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # count since last detection
        self.last_detected = 7
        # count of moving average
        self.detection_count = None
        # x values of the last n fits of the line
        self.recent_xfitted = []
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = [np.array([False])]   
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        #indeces for detected line pixels
        self.all = None

In [ ]:
images = glob.glob('test_images/*.jpg')
M_img = cv2.imread(images[0], cv2.IMREAD_COLOR)
M, Minv = get_warp_matrix(M_img)
ret, mtx, dist, rvecs, tvecs = Calibrate()

In [ ]:
cv2.destroyAllWindows()
# Setup pipeline prequisites
Right_Line = Line()
Left_Line = Line()

images = glob.glob('test_images/*.jpg')
images = [images[1]]
for fname in images:
    image = cv2.imread(fname, cv2.IMREAD_COLOR)
    lanes = process_image(fname,image)
#    cv2.imshow(fname, image)
#    lanes = adv_proc_img(image, movie = False)
#    cv2.imshow(fname, lanes)

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Setup pipeline prequisites
Right_Line = Line()
Left_Line = Line()

# Run lane finding pipeline on video

#video = 'project'
#video = 'challenge'
video = 'harder_challenge'
length = 'full_length'
#length = 'clipped'
comment = '_test_5'
j=0
output = 'output_videos/{}_video_{}{}.mp4'.format(video, length, comment)

if length == 'full_length':
    clip1 = VideoFileClip("{}_video.mp4".format(video))
if length == 'clipped':
    clip1 = VideoFileClip("{}_video.mp4".format(video)).subclip(0,5)
clip = clip1.fl_image(adv_proc_img) #NOTE: this function expects color images!!

%time clip.write_videofile(output, audio=False)

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Setup pipeline prequisites
Right_Line = Line()
Left_Line = Line()

# Run lane finding pipeline on video

#video = 'project'
video = 'challenge'
#video = 'harder_challenge'
length = 'full_length'
#length = 'clipped'
comment = '_test_6'
j=0
output = 'output_videos/{}_video_{}{}.mp4'.format(video, length, comment)

if length == 'full_length':
    clip1 = VideoFileClip("{}_video.mp4".format(video))
if length == 'clipped':
    clip1 = VideoFileClip("{}_video.mp4".format(video)).subclip(0,6)
clip = clip1.fl_image(adv_proc_img) #NOTE: this function expects color images!!

%time clip.write_videofile(output, audio=False)

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Setup pipeline prequisites
Right_Line = Line()
Left_Line = Line()

# Run lane finding pipeline on video

#video = 'project'
#video = 'challenge'
video = 'harder_challenge'
length = 'full_length'
#length = 'clipped'
comment = '_test_4'
j=0
output = 'output_videos/{}_video_{}{}.mp4'.format(video, length, comment)

if length == 'full_length':
    clip1 = VideoFileClip("{}_video.mp4".format(video))
if length == 'clipped':
    clip1 = VideoFileClip("{}_video.mp4".format(video)).subclip(0,6)
clip = clip1.fl_image(adv_proc_img) #NOTE: this function expects color images!!

%time clip.write_videofile(output, audio=False)

In [ ]:
# Excellent frame grabber by Tobi Lehman
# https://tobilehman.com/blog/2013/01/20/extract-array-of-frames-from-mp4-using-python-opencv-bindings/

fname = 'harder_challenge_video.mp4'
vidcap = cv2.VideoCapture(fname)
#The easiest way to extract frames is to use the read() method on the vidcap object.
#It returns a tuple where the first element is a success flag and the second is the image array.
#To extract the image array:
success,image = vidcap.read()
# image is an array of array of [R,G,B] values
#To convert the whole video to frames and save each one:
count = 0; 
frames = 2000
while success:
    success,image = vidcap.read()
    cv2.imwrite("test_video_frames/{}_{}.jpg".format(fname[:-4], count), image)
    if count == frames:
        break
    count += 1